In [1]:
import pandas as pd
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from nbimporter import NotebookLoader
from joblib import load
import statistics
warnings.filterwarnings("ignore")

In [2]:
ma = pd.read_csv("../16 Mycoplosma/ma.csv")
map = pd.read_csv("../16 Mycoplosma/map.csv")
marth = pd.read_csv("../16 Mycoplosma/marth.csv")
mcap = pd.read_csv("../16 Mycoplosma/mcap.csv")
mcon = pd.read_csv("../16 Mycoplosma/mcon.csv")
mcro = pd.read_csv("../16 Mycoplosma/mcro.csv")
mgal = pd.read_csv("../16 Mycoplosma/mgal.csv")
mhom = pd.read_csv("../16 Mycoplosma/mhom.csv")
mhyoJ = pd.read_csv("../16 Mycoplosma/mhyoJ.csv")
mhyo232 = pd.read_csv("../16 Mycoplosma/mhyo232.csv")
mhyo7448 = pd.read_csv("../16 Mycoplosma/mhyo7448.csv")
mm163K = pd.read_csv("../16 Mycoplosma/mm163K.csv")
mms = pd.read_csv("../16 Mycoplosma/mms.csv")
mpHF = pd.read_csv("../16 Mycoplosma/mpHF.csv")
mpM = pd.read_csv("../16 Mycoplosma/mpM.csv")
ms53 = pd.read_csv("../16 Mycoplosma/ms53.csv")

In [3]:
def generate_train_test_dataset(index):
    dataset_names = [ma, map, marth, mcap, mcon, mcro, mgal, mhom, mhyoJ, mhyo232, mhyo7448, mm163K, mms, mpHF, mpM, ms53]
    merged_train = dataset_names[:index]+dataset_names[index+1:]
    train_data = pd.concat(merged_train)
    test_data = dataset_names[index]
    return train_data,test_data

In [4]:
def handle_dataset(dataset):
    columns_to_drop = ['identity','name', 'Amino acid', 'protein_sequence', 'essentiality']
    Xs = dataset.drop(columns = columns_to_drop)
    y = dataset['essentiality']
    return Xs,y

In [5]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [6]:
space = {
    'C': hp.loguniform('C', -5, 2),  # 正则化参数 C
    'degree': hp.quniform('degree', 2, 5, 1),  # 多项式核的次数
    'gamma': hp.choice('gamma', ['auto', 'scale', hp.loguniform('gamma_val', -5, 2)])  # 核函数的系数
}

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LassoCV
def svm_ac_cv(params):
    model = SVC(C = params["C"],kernel ="linear", degree = int(params["degree"]), gamma = params["gamma"])
    scorer = make_scorer(accuracy_score)
    score = -cross_val_score(model, X_resampled, y_resampled, cv=5,scoring=scorer ).mean()
    return score
lassoCV = LassoCV(cv=20,random_state=10)
all_feature_names = ["GC_Content","CAI","A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","nSE2","nSE3","nGE2","nGE3"]

In [8]:
scores = []
score = 0
models = []
for i in range(0,16):
    
    train_data, test_data = generate_train_test_dataset(i)
    Xs_train,y_train = handle_dataset(train_data)
    Xs_test,y_test = handle_dataset(test_data)
    rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
    lassoCV.fit(X_resampled,y_resampled)
    selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
    trials = Trials()
    params=fmin(fn=svm_ac_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=15, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.default_rng(42) # fixing random state for the reproducibility
    )
    model = SVC(C = params["C"],kernel ="linear", degree = int(params["degree"]), gamma = params["gamma"])
    models.append(model)
    model.fit(X_resampled[selected_features],y_resampled)
    tpe_test_score=accuracy_score(y_test, model.predict(Xs_test[selected_features]))
    scores.append(tpe_test_score)
    print(tpe_test_score)
    if score < tpe_test_score:
        score = tpe_test_score
        best_model = model

  0%|          | 0/15 [00:24<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [18]:
print(scores)

[0.8541114058355438, 0.8426501035196687, 0.6823899371069182, 0.9387755102040817, 0.8742331288343558, 0.9322033898305084, 0.6707021791767555, 0.9387096774193548, 0.7861271676300579, 0.9085631349782293, 0.8930635838150289, 0.9366391184573003, 0.942643391521197, 0.945, 0.4392712550607287]


In [21]:
from joblib import dump

dump(best_model, '../../model/SVM_model_lasso.joblib')

['../../model/SVM_model_lasso.joblib']

In [20]:
train_data, test_data = generate_train_test_dataset(12)
Xs_train,y_train = handle_dataset(train_data)
Xs_test,y_test = handle_dataset(test_data)
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
lassoCV.fit(X_resampled,y_resampled)
selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
print(selected_features)

['GC_Content', 'CAI', 'A', 'N', 'D', 'H', 'M', 'nSE3', 'nGE3']
